In [1]:
# Step 1: Import Libraries and Load the Model
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

In [2]:
# Load the pre-trained model with ReLU activation
model = load_model('simple_rnn_imdb.h5')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [3]:
model.get_weights()

[array([[ 1.4459071 ,  0.12807797,  1.4660959 , ...,  0.20180783,
          0.02101618, -1.2885801 ],
        [ 0.13887613, -0.03752521,  0.06508313, ...,  0.09411127,
         -0.02726973,  0.02467358],
        [-0.05278237, -0.11703496,  0.17323793, ...,  0.21676359,
         -0.24681443,  0.10276395],
        ...,
        [ 0.04220016, -0.02516859,  0.14639579, ..., -0.02415404,
          0.00804342, -0.02879291],
        [-0.0288488 ,  0.10898943, -0.01256185, ..., -0.13032861,
         -0.06841935, -0.03675327],
        [ 0.01565286, -0.11984517,  0.13944519, ...,  0.15446155,
          0.18681534,  0.21817695]], dtype=float32),
 array([[-0.0076051 ,  0.03124611,  0.05092288, ...,  0.05134553,
          0.08385516, -0.02715867],
        [ 0.13601324, -0.157263  , -0.09693798, ..., -0.04992546,
          0.07996646,  0.02158053],
        [-0.13830572,  0.10823265, -0.09161095, ...,  0.01884773,
         -0.12188116, -0.03633418],
        ...,
        [-0.15936133,  0.12066077, -0.1

In [4]:
# Assuming model_weights is the list of arrays from model.get_weights()
model_weights = model.get_weights()

# Print the length of the list (number of arrays)
print(f"Number of arrays in the model weights: {len(model_weights)}")

# Print the shape of each array
for i, array in enumerate(model_weights):
    print(f"Shape of array {i + 1}: {array.shape}")


Number of arrays in the model weights: 6
Shape of array 1: (10000, 128)
Shape of array 2: (128, 128)
Shape of array 3: (128, 128)
Shape of array 4: (128,)
Shape of array 5: (128, 1)
Shape of array 6: (1,)


In [5]:
# Sorting the dictionary by values (ascending order)
sorted_word_index = dict(sorted(word_index.items(), key=lambda item: item[1]))


for key, value in sorted_word_index.items():
    if value <= 10:
        print(f"{key}: {value}")

NameError: name 'word_index' is not defined

In [29]:

# Load the IMDB dataset word index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}
reverse_word_index

{34701: 'fawn',
 52006: 'tsukino',
 52007: 'nunnery',
 16816: 'sonja',
 63951: 'vani',
 1408: 'woods',
 16115: 'spiders',
 2345: 'hanging',
 2289: 'woody',
 52008: 'trawling',
 52009: "hold's",
 11307: 'comically',
 40830: 'localized',
 30568: 'disobeying',
 52010: "'royale",
 40831: "harpo's",
 52011: 'canet',
 19313: 'aileen',
 52012: 'acurately',
 52013: "diplomat's",
 25242: 'rickman',
 6746: 'arranged',
 52014: 'rumbustious',
 52015: 'familiarness',
 52016: "spider'",
 68804: 'hahahah',
 52017: "wood'",
 40833: 'transvestism',
 34702: "hangin'",
 2338: 'bringing',
 40834: 'seamier',
 34703: 'wooded',
 52018: 'bravora',
 16817: 'grueling',
 1636: 'wooden',
 16818: 'wednesday',
 52019: "'prix",
 34704: 'altagracia',
 52020: 'circuitry',
 11585: 'crotch',
 57766: 'busybody',
 52021: "tart'n'tangy",
 14129: 'burgade',
 52023: 'thrace',
 11038: "tom's",
 52025: 'snuggles',
 29114: 'francesco',
 52027: 'complainers',
 52125: 'templarios',
 40835: '272',
 52028: '273',
 52130: 'zaniacs',

In [30]:
# self-note: this whole adding 3 to index while predicting,but not adding 3 to index on xtrain while training is confusing, ignore this part and focus on the rest as this is specific to imdb data

# Step 2: Helper Functions
# Function to preprocess user input
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words] # shifting indices by 3 to reserve indices 0,1,2
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review

# Function to decode reviews
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])


In [31]:
### Prediction  function

def predict_sentiment(review):
    preprocessed_input=preprocess_text(review)

    prediction=model.predict(preprocessed_input)

    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    
    return sentiment, prediction[0][0]



In [32]:
# Step 4: User Input and Prediction
# Example review for prediction
example_review = "This movie was fantastic! The acting was great and the plot was thrilling."

sentiment,score=predict_sentiment(example_review)

print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

1/1 [==============================] - 1s 544ms/step
Review: This movie was fantastic! The acting was great and the plot was thrilling.
Sentiment: Positive
Prediction Score: 0.5538839101791382
